In [5]:
import matplotlib.pyplot as plt
import numpy as np
from osgeo import gdal
import numpy.ma as ma


In [6]:
%matplotlib Qt


# years = ['2016', '2017', '2018', '2020', '2021', '2022'] #2023
# num_years = len(years)

# # Define the number of rows and columns for subplots based on the number of years
# num_rows = 1
# num_cols = num_years

# # Create subplots
# fig, axes = plt.subplots(num_rows, num_cols, figsize=(2*num_years, 5),sharex=True, sharey=True)

# # Iterate over each year
# for i, year in enumerate(years):
#     hydroperiod_raster_file = f'/Users/bvarugu/Documents/Belomonte/Xingu_hydroperiod/Xingu_SAR_hydroperiod_{year}.tif'
    
#     # Open the raster file
#     dataset = gdal.Open(hydroperiod_raster_file)
    
#     # Check if the raster file is successfully opened
#     if dataset is None:
#         print(f"Failed to open raster file for year {year}")
#         continue
    
#     # Read the raster band data
#     hydroperiod = dataset.GetRasterBand(1).ReadAsArray().astype(float)
#     # Close the dataset
#     dataset = None
    
#     # Plot the binary mask
#     axes[i].imshow(np.ma.masked_where(hydroperiod <0, hydroperiod), cmap='Blues', vmin=0, vmax=1)
#     axes[i].set_title(str(year)+'_hydroperiod')
#     axes[i].axis('off')

# # Adjust layout and display plot
# plt.tight_layout()
# plt.show()

In [17]:

years = ['2016', '2018', '2020', '2022']
#years = ['2017', '2019', '2021', '2023']
#years = ['2016', '2017','2018','2019', '2020','2021', '2022']
num_years = len(years)
LW_mask_dict = {}

# Define a helper function to read and process the raster file
def process_raster_file(file_path,thres):
    dataset = gdal.Open(file_path)
    if dataset is None:
        print(f"Failed to open raster file {file_path}")
        return None, None
    band = dataset.GetRasterBand(1)
    hydroperiod = band.ReadAsArray().astype(float)
    nodata_value = band.GetNoDataValue();
    metadata = dataset.GetMetadata(); # if using rasterio, metadata = dataset.tags()
    
    # If 'dates' is part of the metadata, split it into a list
    if 'dates' in metadata:
        metadata['dates'] = metadata['dates'].split(',');
#         if len(metadata['dates'])>10:
#             thres = np.floor()
        print(metadata['dates'])
    dataset = None
    binary_mask = np.zeros(hydroperiod.shape, dtype=np.uint8)
    binary_mask[hydroperiod < thres] = 1
    if nodata_value is not None:
        binary_mask[hydroperiod == nodata_value] = 255
    return binary_mask, hydroperiod

for year in years:
    hydroperiod_threshold = 0.75;
    hydroperiod_raster_file = f'/Users/bvarugu/Documents/Belomonte/Xingu_hydroperiod/new/Xingu_SAR_hydroperiod_{year}_new.tif'
    binary_mask, hydroperiod = process_raster_file(hydroperiod_raster_file,hydroperiod_threshold)
    if binary_mask is not None:
        LW_mask_dict[year] = {
            'mask': binary_mask,
            'hydroperiod': hydroperiod
        }

# fig, axes = plt.subplots(2, len(years), figsize=(15, 6), sharex=True, sharey=True)

# for i, year in enumerate(years):
#     mask = LW_mask_dict[year]['mask']
    
#     if i == 0:
#         axes[0, i].imshow(ma.masked_where(mask == 255, mask), cmap='jet', vmin=0, vmax=2,interpolation='nearest')
#         axes[0, i].set_title(f'Year {year}')
#     else:
#         prev_mask = LW_mask_dict[years[i-1]]['mask']
#         change_mask = mask.astype(int) - prev_mask.astype(int)
#         change_mask = ma.masked_where(change_mask == 0, change_mask)
#         axes[0, i].imshow(ma.masked_where(mask == 255, change_mask), cmap='coolwarm', vmin=-1, vmax=1,interpolation='nearest')
#         num_positive_changes = np.sum(change_mask == 1)
#         num_negative_changes = np.sum(change_mask == -1)
        
#         axes[0, i].set_title(f'Change {years[i-1]} to {year}\n(+): {num_positive_changes} (-): {num_negative_changes}')
    
    
#     axes[1, i].imshow(ma.masked_where(mask == 255, mask), cmap='jet', vmin=0, vmax=2,interpolation='nearest')
#     axes[1, i].set_title(f'Year {year}')
#     axes[0,i].axis('off');axes[1,i].axis('off');

# plt.tight_layout()
# plt.show()

['20160929', '20161011', '20161023', '20161104', '20161116', '20161128', '20161210', '20161222']
['20180802', '20180814', '20180826', '20180907', '20180919', '20181001', '20181013', '20181025', '20181106', '20181118', '20181130', '20181212', '20181224']
['20200815', '20200827', '20200908', '20200920', '20201002', '20201026', '20201107', '20201119', '20201201', '20201213', '20201225']
['20220817', '20220829', '20220910', '20220922', '20221004', '20221016', '20221028', '20221109', '20221121', '20221203', '20221215', '20221227']


In [18]:
fig, axes = plt.subplots(2, len(years), figsize=(15, 6), sharex=True, sharey=True)
initial_mask = LW_mask_dict[years[0]]['mask'];
for i, year in enumerate(years):
    mask = LW_mask_dict[year]['mask']
    
    if i == 0:
        axes[0, i].imshow(ma.masked_where(mask == 255, mask), cmap='jet', vmin=0, vmax=2,interpolation='nearest')
        axes[0, i].set_title(f'Year {year}');
        cumm_change_mask = np.zeros(np.shape(mask));
    else:
        prev_mask = LW_mask_dict[years[i-1]]['mask']
        change_mask = mask.astype(int) - prev_mask.astype(int);
        cumm_change_mask += change_mask;
        axes[0, i].imshow(ma.masked_where(initial_mask == 255, initial_mask), cmap='Blues_r', vmin=-1, vmax=1,interpolation='nearest')
        axes[0, i].imshow(ma.masked_where(cumm_change_mask == 0, cumm_change_mask), cmap='PiYG', vmin=-1, vmax=1,interpolation='nearest')
        num_positive_changes = np.sum(change_mask == 1)
        num_negative_changes = np.sum(change_mask == -1)
        
        axes[0, i].set_title(f'Change {years[i-1]} to {year}\n(+): {num_positive_changes} (-): {num_negative_changes}')
    
    
    axes[1, i].imshow(ma.masked_where(mask == 255, mask), cmap='jet', vmin=0, vmax=2,interpolation='nearest')
    axes[1, i].set_title(f'Year {year}')
    axes[0,i].axis('off');axes[1,i].axis('off');

plt.tight_layout()
plt.show()

## Compare all the changes with respect to 2016
### create change mask by subtracting from 2016 mask

In [9]:
fig, axes = plt.subplots(2, len(years), figsize=(15, 6), sharex=True, sharey=True);
initial_mask = LW_mask_dict[years[0]]['mask'];
for i, year in enumerate(years):
    mask = LW_mask_dict[year]['mask']
    
    if i == 0:
        axes[0, i].imshow(ma.masked_where(initial_mask == 255, initial_mask), cmap='Blues_r', vmin=-1, vmax=1,interpolation='nearest')
        axes[0, i].set_title(f'Year {year}');
        cumm_change_mask = np.zeros(np.shape(mask));
    else:
        #prev_mask = LW_mask_dict[years[0]]['mask']
        change_mask = mask.astype(int) - initial_mask.astype(int);
        cumm_change_mask += change_mask;
        axes[0, i].imshow(ma.masked_where(initial_mask == 255, initial_mask), cmap='Blues_r', vmin=-1, vmax=1,interpolation='nearest')
        axes[0, i].imshow(ma.masked_where(cumm_change_mask == 0, cumm_change_mask), cmap='PiYG', vmin=-(len(years)-1), vmax=(len(years)-1),interpolation='nearest')
        num_positive_changes = np.sum(change_mask == 1)
        num_negative_changes = np.sum(change_mask == -1)
        
        axes[0, i].set_title(f'Change {years[i-1]} to {year}\n(+): {num_positive_changes} (-): {num_negative_changes}')
    
    
    axes[1, i].imshow(ma.masked_where(mask == 255, mask), cmap='Blues_r', vmin=-1, vmax=1,interpolation='nearest')
    axes[1, i].set_title(f'Year {year}')
    axes[0,i].axis('off');axes[1,i].axis('off');

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, len(years), figsize=(15, 6), sharex=True, sharey=True);
initial_mask = LW_mask_dict[years[0]]['mask'];
for i, year in enumerate(years):
    mask = LW_mask_dict[year]['mask']
    
    if i == 0:
        axes[0, i].imshow(ma.masked_where(initial_mask == 255, initial_mask), cmap='Blues_r', vmin=-1, vmax=1,interpolation='nearest')
        axes[0, i].set_title(f'Year {year}');
    else:
        #prev_mask = LW_mask_dict[years[0]]['mask']
        change_mask = mask.astype(int) - initial_mask.astype(int);
        axes[0, i].imshow(ma.masked_where(initial_mask == 255, initial_mask), cmap='Blues_r', vmin=-1, vmax=1,interpolation='nearest')
        axes[0, i].imshow(ma.masked_where(change_mask == 0, change_mask), cmap='PiYG', vmin=-2, vmax=2,interpolation='nearest')
        #num_positive_changes = np.sum(change_mask == 1)
        #num_negative_changes = np.sum(change_mask == -1)
        
        #axes[0, i].set_title(f'Change {years[i-1]} to {year}\n(+): {num_positive_changes} (-): {num_negative_changes}')
    
    
    axes[1, i].imshow(ma.masked_where(mask == 255, mask), cmap='Blues_r', vmin=-1, vmax=1,interpolation='nearest')
    axes[1, i].set_title(f'Year {year}')
    axes[0,i].axis('off');axes[1,i].axis('off');

plt.tight_layout()
plt.show()

In [7]:
initial_mask = LW_mask_dict[years[0]]['mask']
cumm_change_mask = np.zeros_like(initial_mask, dtype=int)

positive_changes = []
negative_changes = []

for i in range(1, len(years)):
    year = years[i]
    mask = LW_mask_dict[year]['mask']
    
    change_mask = mask.astype(int) - initial_mask.astype(int)
    cumm_change_mask += change_mask
    
    num_positive_changes = np.sum(cumm_change_mask > 0)
    num_negative_changes = np.sum(cumm_change_mask < 0)
    
    positive_changes.append(num_positive_changes*100)
    negative_changes.append(num_negative_changes*100)

# Create a bar plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.tick_params(labelsize=15)
bar_width = 0.35
index = np.arange(len(years) - 1)

bar1 = ax.bar(index, np.array(positive_changes)*1e-6, bar_width, label='Erosion', color='Blue')
bar2 = ax.bar(index, -1*np.array(negative_changes)*1e-6, bar_width, label='Accretion', color='Orange')

ax.set_xlabel('Year',fontsize=15)
ax.set_ylabel('Area in sq.km',fontsize=15)
ax.set_title('Cumulative Erosion and Accetion Change w.r.t 2016')
ax.set_xticks(index)
ax.set_xticklabels(years[1:])
ax.set_ylim([-np.max(positive_changes)*1.1*1e-6,np.max(positive_changes)*1.1*1e-6])
ax.legend()

plt.tight_layout()
plt.show()

In [8]:

positive_changes = []
negative_changes = []

for i in range(1, len(years)):
    year2 = years[i];year1 = years[i-1];
    mask2 = LW_mask_dict[year2]['mask'];
    #mask1 = LW_mask_dict[year1]['mask'];
    mask1 = LW_mask_dict['2016']['mask']
    
    change_mask = mask2.astype(int) - mask1.astype(int)
    
    num_positive_changes = np.sum(change_mask > 0)
    num_negative_changes = np.sum(change_mask < 0)
    
    positive_changes.append(num_positive_changes*100)
    negative_changes.append(num_negative_changes*100)

# Create a bar plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.tick_params(labelsize=15)
bar_width = 0.35
index = np.arange(len(years) - 1)

bar1 = ax.bar(index, np.array(positive_changes)*1e-6, bar_width, label='Erosion', color='Blue')
bar2 = ax.bar(index, -1*np.array(negative_changes)*1e-6, bar_width, label='Accretion', color='Orange')

ax.set_xlabel('Year',fontsize=15)
ax.set_ylabel('Area in sq.km',fontsize=15)

#ax.set_title('Relative to previous year Erosion and Accetion Changes')
ax.set_title('Relative to 2016 Erosion and Accetion Changes')
ax.set_xticks(index)
ax.set_xticklabels(years[1:])
ax.set_ylim([-np.max(positive_changes)*1.1*1e-6,np.max(positive_changes)*1.1*1e-6])
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
years = ['2016', '2017', '2018', '2019','2020', '2022'] #2023
num_years = len(years);
LW_mask_dict= {}
for i, year in enumerate(years):
    hydroperiod_raster_file = f'/Users/bvarugu/Documents/Belomonte/Xingu_hydroperiod/Xingu_SAR_hydroperiod_{year}.tif'
    
    # Open the raster file
    dataset = gdal.Open(hydroperiod_raster_file)
    
    # Check if the raster file is successfully opened
    if dataset is None:
        print(f"Failed to open raster file for year {year}")
        continue
    
    # Read the raster band data
    band = dataset.GetRasterBand(1)
    hydroperiod = band.ReadAsArray().astype(float)
    nodata_value = band.GetNoDataValue()
    # Close the dataset
    dataset = None
    binary_mask = np.zeros(hydroperiod.shape, dtype=np.uint8)
    binary_mask[hydroperiod < 1] = 1  # Apply threshold
    #binary_mask[hydroperiod==nodata_value] = 255;
    LW_mask_dict[year] = {
        'mask': binary_mask,
        'hydroperiod': hydroperiod
        # Add any other relevant metadata
    }


fig, axes = plt.subplots(2, len(years), figsize=(10, 5),sharex=True, sharey=True);

for i in range(len(years)):
    if i==0:
        axes[0,i].imshow(np.ma.masked_where(LW_mask_dict[years[i]]['mask']==255, LW_mask_dict[years[i]]['mask']), cmap='jet', vmin=0, vmax=2);
    else:
        mask1= LW_mask_dict[years[i]]['mask'];
        mask2= LW_mask_dict[years[i-1]]['mask'];
        change_mask2 = mask1-mask2;
        axes[0,i].imshow(change_mask2, cmap='jet', vmin=-1, vmax=1);
        del change_mask2
    axes[1,i].imshow(np.ma.masked_where(LW_mask_dict[years[i]]['mask']==255, LW_mask_dict[years[i]]['mask']), cmap='jet', vmin=0, vmax=2);  

In [ ]:
%matplotlib Qt
years2= ['2016','2018','2020','2022']
num_cols = len(years2)
fig, axes = plt.subplots(2, num_cols, figsize=(10, 5),sharex=True, sharey=True);

for i, year in enumerate(years2):
    hydroperiod = LW_mask_dict[year]['hydroperiod'];
    mask = LW_mask_dict[year]['mask'];
    axes[1,i].imshow(np.ma.masked_where(mask==255, mask), cmap='jet', vmin=0, vmax=2)
    axes[0,i].imshow(np.ma.masked_where(hydroperiod <0, hydroperiod), cmap='Blues', vmin=0, vmax=1)
    axes[1,i].set_title(str(year)+'_LW_mask');axes[0,i].set_title(str(year)+'_hydroperiod')
    axes[0,i].axis('off');axes[1,i].axis('off');

In [ ]:
previous_mask = LW_mask_dict[years[0]]['mask'];
net_change = np.zeros_like(previous_mask, dtype=int)
    
    # Process each mask
for i,year in enumerate(years[1:]):
    print(year)
    current_mask = LW_mask_dict[year]['mask'];

    # Compute land gain and loss
    land_gain = np.logical_and(current_mask == 1, previous_mask == 0).astype(int)
    land_loss = np.logical_and(current_mask == 0, previous_mask == 1).astype(int)

    # Update net change array
    net_change += land_gain
    net_change -= land_loss

    # Update previous mask
    previous_mask = current_mask

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 5),sharex=True, sharey=True);
axes[0].imshow(np.ma.masked_where(LW_mask_dict['2016']['mask']==255, LW_mask_dict['2016']['mask']), cmap='Blues_r', vmin=0, vmax=1,interpolation='nearest');
axes[1].imshow(np.ma.masked_where(net_change==0, net_change), cmap='RdBu_r', vmin=-2, vmax=2,interpolation='nearest')
axes[2].imshow(np.ma.masked_where(LW_mask_dict['2022']['mask']==255, LW_mask_dict['2022']['mask']), cmap='Blues_r', vmin=0, vmax=1,interpolation='nearest')
axes[0].axis('off');axes[1].axis('off');axes[2].axis('off');
axes[0].set_title('2016_LW_mask');
axes[1].set_title('Net Land Gain/Loss');
axes[2].set_title('2022_LW_mask');



In [ ]:
previous_mask = LW_mask_dict[years[0]]['mask'];
net_change = np.zeros_like(previous_mask, dtype=int);

land_gain_list = [];
land_loss_list = [];
    
    # Process each mask
for year in years[1:]:
    print(year)
    current_mask = LW_mask_dict[year]['mask'];

    # Compute land gain and loss
    land_gain = np.logical_and(current_mask == 1, previous_mask == 0).astype(int)
    land_loss = np.logical_and(current_mask == 0, previous_mask == 1).astype(int)

    # Update net change array
    net_change += land_gain
    net_change -= land_loss

    # Update previous mask
    previous_mask = current_mask